In [2]:
import pandas as pd
import pyodbc

In [11]:
# SQL Server bağlantısı
conn = pyodbc.connect(

    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=SEMRA\\SQLEXPRESS;"
    "DATABASE=PharmaSalesDB;"
    "Trusted_Connection=yes;"
)

In [14]:
# Core tablosunu oku
df = pd.read_sql("SELECT * FROM core.PharmaceuticalSales;", conn)

# Veriye göz atalım
df.head()

C:\Users\semra\AppData\Local\Temp\ipykernel_30896\109088849.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM core.PharmaceuticalSales;", conn)


,sale_id,date,company,product,atc_class,rx_otc,form,strength_mg,package_size,channel,...,city,sales_rep,doctor_specialty,units_sold,unit_price_try,discount_rate,gross_revenue_try,net_revenue_try,prescription_count,patient_age_group
0,257,2024-01-01,Abdi Ibrahim,Ventolin HFA,R03AC02 (SABA),RX,Inhaler,100,1,Eczane,...,Mardin,SR-030,Enfeksiyon,19,281.25,0.030,5343.75,5183.44,9,41-64
1,198,2024-01-03,Novartis TR,Paraset,N02BE01 (Analgesic),OTC,Tablet,500,20,Hastane,...,Zonguldak,SR-007,Enfeksiyon,20,72.13,0.197,1442.60,1158.41,0,19-40
2,148,2024-01-03,Bayer TR,Ventolin HFA,R03AC02 (SABA),RX,Inhaler,100,1,Eczane,...,Eskişehir,SR-012,Göğüs Hastalıkları,15,273.22,0.110,4098.30,3647.49,5,13-18
3,146,2024-01-06,Bayer TR,Xarelix,B01AF01 (DOAC),RX,Tablet,20,28,Hastane,...,Şanlıurfa,SR-007,Göğüs Hastalıkları,69,808.06,0.057,55756.14,52578.04,22,19-40
4,35,2024-01-07,Pfizer TR,Metfor,A10BA02 (Metformin),RX,Tablet,850,60,Hastane,...,Mersin,SR-023,Pediatri,72,105.79,0.066,7616.88,7114.17,20,19-40


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sale_id             300 non-null    int64  
 1   date                300 non-null    object 
 2   company             300 non-null    object 
 3   product             300 non-null    object 
 4   atc_class           300 non-null    object 
 5   rx_otc              300 non-null    object 
 6   form                300 non-null    object 
 7   strength_mg         300 non-null    int64  
 8   package_size        300 non-null    int64  
 9   channel             300 non-null    object 
 10  region              300 non-null    object 
 11  city                300 non-null    object 
 12  sales_rep           270 non-null    object 
 13  doctor_specialty    271 non-null    object 
 14  units_sold          300 non-null    int64  
 15  unit_price_try      300 non-null    float64
 16  discount

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Aylık toplam net gelir
df["date"] = pd.to_datetime(df["date"])
monthly = (
    df.assign(month=df["date"].dt.to_period("M").dt.to_timestamp())
      .groupby("month", as_index=False)["net_revenue_try"].sum()
      .rename(columns={"net_revenue_try": "net_revenue"})
      .sort_values("month")
)

# Zaman değişkeni (trend)
monthly["t"] = np.arange(len(monthly))

# Modeli kur
X = monthly[["t"]]
y = monthly["net_revenue"]
model = LinearRegression().fit(X, y)

# Tahmin çizgisi
monthly["pred"] = model.predict(X)

# 1–3 ay ileri forecast
last_t = monthly["t"].iloc[-1]
future = pd.DataFrame({"t": [last_t+1, last_t+2, last_t+3]})
future["forecast"] = model.predict(future[["t"]])
last_month = monthly["month"].iloc[-1]
future["month"] = [last_month + pd.offsets.MonthBegin(i) for i in [1,2,3]]

# Çıktılar
print("✨ 1–3 Ay İleri Tahmin ✨")
for m, f in zip(future["month"], future["forecast"]):
    print(m.date(), "→", round(float(f), 2), "TRY")

# Power BI için kaydet
out = monthly[["month", "net_revenue", "pred"]]
out.to_csv("pharma_forecast_baseline.csv", index=False)
print("\nKaydedildi: pharma_forecast_baseline.csv")


✨ 1–3 Ay İleri Tahmin ✨
2025-10-01 → 109395.1 TRY
2025-11-01 → 111286.37 TRY
2025-12-01 → 113177.65 TRY

Kaydedildi: pharma_forecast_baseline.csv


In [29]:
from sklearn.metrics import r2_score, mean_absolute_error

# Model performansı (tam veri üzerinde)
r2 = r2_score(y, monthly["pred"])
mae = mean_absolute_error(y, monthly["pred"])

print("\nModel Performansı:")
print(f"R2: {r2:.3f}")
print(f"MAE: {mae:.2f} TRY")


Model Performansı:
R2: 0.077
MAE: 34262.05 TRY


In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# 1️⃣ Aylık toplam net gelir + ortalama indirim
df["date"] = pd.to_datetime(df["date"])
monthly = (
    df.assign(month=df["date"].dt.to_period("M").dt.to_timestamp())
      .groupby("month", as_index=False)
      .agg(net_revenue=("net_revenue_try", "sum"),
           avg_discount=("discount_rate", "mean"))
      .sort_values("month")
)

# 2️⃣ Zaman ve mevsim değişkenleri
monthly["t"] = np.arange(len(monthly))              # zaman trendi
monthly["month_no"] = monthly["month"].dt.month     # mevsimsellik (1-12)

# 3️⃣ Özellik seti (X) ve hedef (y)
X = monthly[["t", "month_no", "avg_discount"]]
y = monthly["net_revenue"]

# 4️⃣ Model
model = LinearRegression().fit(X, y)
monthly["pred"] = model.predict(X)

# 5️⃣ Model performansı
r2 = r2_score(y, monthly["pred"])
mae = mean_absolute_error(y, monthly["pred"])
print(f"R2: {r2:.3f}")
print(f"MAE: {mae:.2f} TRY")

# 6️⃣ 1–3 ay ileri tahmin
last_t = monthly["t"].iloc[-1]
last_month = monthly["month"].iloc[-1]
future = pd.DataFrame({
    "t": [last_t+1, last_t+2, last_t+3],
    "month_no": [(last_month + pd.offsets.MonthBegin(i)).month for i in [1,2,3]],
    "avg_discount": [monthly["avg_discount"].mean()]*3   # basit varsayım: ortalama indirim sabit
})
future["forecast"] = model.predict(future[["t", "month_no", "avg_discount"]])
future["month"] = [last_month + pd.offsets.MonthBegin(i) for i in [1,2,3]]

print("\n✨ 1–3 Ay İleri Tahmin ✨")
for m, f in zip(future["month"], future["forecast"]):
    print(m.date(), "→", round(float(f), 2), "TRY")

# 7️⃣ Power BI için dosya kaydet
out = monthly[["month", "net_revenue", "pred", "avg_discount"]]
out.to_csv("pharma_forecast_multi_trend.csv", index=False)
print("\nKaydedildi: pharma_forecast_multi_trend.csv")

R2: 0.230
MAE: 29378.56 TRY

✨ 1–3 Ay İleri Tahmin ✨
2025-10-01 → 105330.47 TRY
2025-11-01 → 107115.37 TRY
2025-12-01 → 108900.27 TRY

Kaydedildi: pharma_forecast_multi_trend.csv
